In [1]:
#Import Packages. 
import sys
import os
import requests
import warnings
import glob
import time
import re
from functools import reduce

import matplotlib.pyplot as plt
import seaborn as sns
import numpy.ma as ma
import numpy as np
#from shapely.geometry import mapping, box
import geopandas as gpd
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
from osgeo import gdal
import pandas as pd

#GFS data
import xarray # used for reading the data.
import xarray_extras.csv # used for writing data to csv format. 
import pygrib
import xarray # used for reading the data.
import ipywidgets as widgets
import matplotlib.pyplot as plt # used to plot the data.
import ipywidgets as widgets # For ease in selecting variables.
import cartopy.crs as ccrs # Used to georeference data.

#For writing to s3
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import io
import pickle

#For timing function
from profilehooks import profile

#For multiprocessing of function
import multiprocessing
from multiprocessing import Pool


# #from cloudpathlib import S3Path, S3Client
# from pyhdf.SD import SD, SDC

warnings.simplefilter('ignore')

In [2]:
s3 = boto3.client('s3')

In [3]:
df_grids = pd.read_csv('../capstone/grid_metadata.csv')

final_grid_ids = []
final_min_lats = []
final_max_lats = []
final_min_lons = []
final_max_lons = []

#Note order is lon, lat for each of the comma separated values
for index, row in df_grids.iterrows(): 
    lons = []
    lats = []
    grid_id = row['grid_id']
    nums = row['wkt'][10:-2]
    nums = nums.replace(',','')
    pairs = nums.split(' ')
    for i in range(10): 
        if i % 2 == 0: 
            lons.append(pairs[i])
        else: 
            lats.append(pairs[i])
    
    #If negative the max negative number is actually lower. 
    if float(max(lons)) < 0: 
        min_lon = float(max(lons)) - 0.15
        max_lon = float(min(lons)) + 0.15
    else: 
        min_lon = float(min(lons)) - 0.15
        max_lon = float(max(lons)) + 0.15
    
    # Adding +/- 0.125 guarantees we get at least 1, 0.25 degree forecast in the appropriate area (given GFS scale is larger than Sat. scale).      
    min_lat = float(min(lats)) - 0.15
    max_lat = float(max(lats)) + 0.15
    
    #Now append the appropriate scraped data to our lists to put into a dataframe. 
    final_grid_ids.append(grid_id)
    final_min_lats.append(min_lat)
    final_max_lats.append(max_lat)
    final_min_lons.append(min_lon)
    final_max_lons.append(max_lon) 
                        
    
    
df_grids_clean = pd.DataFrame(columns = ['grid_id', 'min_lat', 'max_lat', 'min_lon', 'max_lon'])


df_grids_clean['grid_id'] = final_grid_ids 
df_grids_clean['min_lat'] = final_min_lats
df_grids_clean['max_lat'] = final_max_lats
df_grids_clean['min_lon'] = final_min_lons
df_grids_clean['max_lon'] = final_max_lons

    
df_grids_clean

,grid_id,min_lat,max_lat,min_lon,max_lon
0,1X116,24.827661,25.168369,121.330849,121.675764
1,1Z2W7,28.396645,28.736092,77.109616,77.454532
2,3S31A,33.645584,33.982902,-118.128741,-117.783825
3,6EIL6,28.396645,28.736092,76.885037,77.229953
4,7334C,28.396645,28.736092,76.929953,77.274869
5,78V83,28.396645,28.736092,76.750290,77.095206
6,7F1D1,28.436092,28.775526,76.929953,77.274869
7,8KNI6,28.317704,28.657182,77.109616,77.454532
8,90BZ1,24.868369,25.209064,121.375764,121.720680
9,90S79,28.475526,28.814944,77.019784,77.364700


In [ ]:
url = 'https://opendap.nccs.nasa.gov/dods/gmao/geos-cf/assim/aqc_tavg_1hr_g1440x721_v1'
DATASET = xarray.open_dataset(url)
s_type = ['pm25_rh35_gcc','co']
# start_time = np.datetime64('2018-02-01 00:00:00')
# end_time = np.datetime64('2018-03-01 00:00:00')
# minimum_latitude = 33.646149
# minumum_longitude = -117.349920
# maximum_latitude = 33.683532
# maximum_longitude = -117.305004
# DATA_SUBSET = DATASET[s_type].loc[{'time':slice(start_time,end_time),'lat':slice(minimum_latitude,maximum_latitude),'lon':slice(minumum_longitude,maximum_longitude)}]

In [4]:
#years = ['2018', '2019', '2020', '2021']
#months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

years = ['2018']
months = ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [ ]:
for i_year in range(len(years)): 
    for i_month in range(len(months) - 1): 
        url = 'https://opendap.nccs.nasa.gov/dods/gmao/geos-cf/assim/aqc_tavg_1hr_g1440x721_v1'
        DATASET = xarray.open_dataset(url)
        s_type = ['pm25_rh35_gcc','co', 'no2', 'o3', 'so2']
        start_time = np.datetime64(years[i_year] + '-' + months[i_month] + '-01 00:00:00')
        end_time = np.datetime64(years[i_year] + '-' + months[i_month + 1] + '-01 00:00:00')
        for index, row in df_grids_clean.iterrows(): 
            grid_id = row['grid_id']
            min_lat = row['min_lat']
            max_lat = row['max_lat']
            min_lon = row['min_lon']
            max_lon = row['max_lon']


            DATA_SUBSET = DATASET[s_type].loc[{'time':slice(start_time,end_time),'lat':slice(min_lat,max_lat),'lon':slice(min_lon,max_lon)}]

            df = DATA_SUBSET.to_dataframe()
            df.insert(0, 'grid_id', grid_id)


            out_parquet =  'geo.0p25.' + grid_id + '_' + years[i_year] +  months[i_month] +'.parquet'
            filepath = '../geos/train/' + out_parquet
            df.to_parquet(path = filepath, engine = 'pyarrow')

            #Put file in read mode so we can upload to s3 / Databricks storage bucket. 
            with open(filepath, 'rb') as data:
                s3.upload_fileobj(data, 'capstone-particulate-storage', out_parquet)

In [ ]:
test_df = pd.read_parquet('../geos/train/geo.0p25.1X116_201802.parquet', engine='pyarrow')
test_df